### Subword Tokenizer

| **Tokenizer 방식** | **토큰 단위**                      | **vocab size** | **미등록 단어에 대한 가정**                                                                                  |
|---------------------|------------------------------------|----------------|-------------------------------------------------------------------------------------------------------------|
| **사전 기반**       | 알려진 단어/형태소의 결합           | unlimited       | - 알려진 단어/형태소의 결합이라고 가정<br>- 필요한 형태소 분석 가능<br>- 사전에 등록되지 않은 단어는 UNK 처리 |
| **sub-word**        | 알려진 글자 및 sub-word            | fixed           | - 알려진 sub-words로 분해<br>- 예: appear → app + ear<br>- 자주 등장하는 단어를 제대로 인식 가능<br>- UNK의 개수 최소화 |

- wordpice 방식 : 입력되는 모든 단어에 대해서 작은 단위로 쪼갠 후 빈도가 높은 조합들간의 확률이 높은 단어들로 단어사전을 구축한다.
- BPE(Byte Pair Encoding) 방식 : 문자의쌍을 병합하는 방식


In [ ]:
# 네이버 영화 리뷰 데이터
import urllib.request
import os

# 파일을 저장하는 함수
def get_file(filename, origin_url):
    # expanduser() : ~를 사용자 홈 디렉토리로 변환
    cache_dir = os.path.expanduser('~/.torch/datasets/') # 캐시 디렉토리 경로
    os.makedirs(cache_dir, exist_ok=True) # 디렉토리 생성 (존재하지 않을 경우)
    filepath = os.path.join(cache_dir, filename) # 전체 파일 경로
    
    # exists() : 파일이나 디렉토리가 존재하는지 확인
    if not os.path.exists(filepath): # 파일이 존재하지 않을 경우 다운로드
        print(f'Downloading {origin_url} to {filepath}') # 다운로드 메시지 출력
        urllib.request.urlretrieve(origin_url, filepath) # 파일 다운로드
        
    return filepath

In [4]:
# nsmc 데이터 셋 일부 - 네이버 영화 리뷰 데이터 다운로드
# 훈련 데이터와 테스트 데이터의 경로를 가져옴 
ratings_train_path  = get_file('ratings_train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
ratings_test_path  = get_file('ratings_test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

print(ratings_train_path, ratings_test_path)

C:\Users\roxie/.torch/datasets/ratings_train.txt C:\Users\roxie/.torch/datasets/ratings_test.txt


In [5]:
import pandas as pd

# csv 형식으로 읽지만, 탭(tab)으로 구분된 파일임을 지정
ratings_train_path_df = pd.read_csv(ratings_train_path, sep='\t')  
display(ratings_train_path_df.head()) 

ratings_test_path_df = pd.read_csv(ratings_test_path, sep='\t')  
display(ratings_test_path_df.head())  

id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1

id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0

In [6]:
ratings_train_path_df.isna().sum()  # train 데이터 결측치 확인

id          0
document    5
label       0
dtype: int64

In [7]:
ratings_test_path_df.isna().sum()  # test 데이터 결측치 확인

id          0
document    3
label       0
dtype: int64

In [8]:
# 결측치 제거
# - 결측치가 하나라도 있으면 해당 행 제거
ratings_train_path_df = ratings_train_path_df.dropna(how='any')
ratings_test_path_df = ratings_test_path_df.dropna(how='any')  

ratings_train_path_df.shape, ratings_test_path_df.shape

((149995, 3), (49997, 3))

In [ ]:
# txt 파일 생성 - 학습 데이터
with open('naver_review.txt', 'w', encoding='utf-8') as f:
    for doc in ratings_train_path_df['document'].values: # 'document' 열의 값들을 순회
        f.write(doc + "\n")

### Sentence Piece Tokenizer
- BPE와 Unigram Language Module 두가지 알고리즘을 지원한다.
- 단어 경계에 의존하지 않으므로, 공백도 하나의 심볼로 취급한다.
- 가장 활용도가 높음.
- C++을 기반으로 만들어져 있기 때문에 커멘드(cmd)로 학습 시킨다.
- 문장 전체를 하나의 문자열로 본다.(공백을 포함하여 작은 단위로 토큰화)

In [10]:
!pip install sentencepiece

In [ ]:
import sentencepiece as stp

input = 'naver_review.txt' # 입력 파일
vocab_size = 10000 # 단어 집합의 크기
model_prefix = 'naver_review' # 모델 파일의 접두사
cmd = f'--input={input} --model_prefix={model_prefix} --vocab_size={vocab_size}' # 학습 명령어

stp.SentencePieceTrainer.Train(cmd)     # 모델 학습 후 저장

# 결과 : naver_review.model, naver_review.vocab 파일이 생성됨
# - naver_review.model : 학습된 모델 파일
# - naver_review.vocab : 단어 집합 파일


In [13]:
######## naver_review.model 파일로 토큰화 ########

# SentencePieceProcessor() : sentencepiece 토큰화 객체 생성
sp = stp.SentencePieceProcessor()

# Load() : 학습된 모델 파일 로드
sp.Load(f'{model_prefix}.model')

# 'document' 열의 값들을 순회 (처음 3개)
for doc in ratings_train_path_df['document'].values[:3]:  
    print('원문 :', doc)  # 원문 출력
    print('토큰화 :', sp.encode_as_pieces(doc))  # EncodeAsPieces:  토큰화 결과 출력
    print('인덱스 :', sp.encode_as_ids(doc))  # EncodeAsIds: 토큰 인덱스 출력
    print()
    
    # [ 결과 ]
    # encode_as_pieces : 
    #    - 유니코드 형식으로 토큰화(공백까지 문자로 인식하여 언더바로 표시)
    #    - 결과가 문자로 출력됨
    # encode_as_ids : 
    #    - vocab(단어사전) 파일에 기반하여 토큰을 인덱스로 변환
    #    - 각 토큰에 대한 고유 인덱스 부여


원문 : 아 더빙.. 진짜 짜증나네요 목소리
토큰화 : ['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']
인덱스 : [62, 877, 5, 31, 2019, 68, 1710]

원문 : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
토큰화 : ['▁흠', '...', '포스터', '보고', '▁초딩', '영화', '줄', '....', '오', '버', '연기', '조차', '▁가볍지', '▁않', '구나']
인덱스 : [1634, 8, 4908, 159, 1460, 33, 264, 60, 173, 548, 410, 1224, 7396, 754, 440]

원문 : 너무재밓었다그래서보는것을추천한다
토큰화 : ['▁너무', '재', '밓', '었다', '그래서', '보', '는것을', '추천', '한다']
인덱스 : [23, 369, 9781, 429, 3780, 143, 6266, 1945, 314]



In [15]:
# 단어 집합의 크기 확인(= vocab_size)
sp.get_piece_size()  
sp.GetPieceSize() 

# 두가지 메서드 모두 단어 집합의 크기를 반환한다.
# get_piece_size() : 소문자 p
# GetPieceSize() : 대문자 P
# - 두 메서드 모두 동일한 기능을 수행함

10000

In [32]:
# 인코딩
text = ratings_test_path_df['document'][100]  # 테스트 데이터의 100 번째 문장
print('원문 :', text)

tokens = sp.encode_as_pieces(text) # 토큰화 (텍스트 -> subword 단위 분할)
id_tokens = sp.encode_as_ids(text) # 토큰의 인덱스 (고유 ID 변환)

print('토큰화 :', tokens)
print('인덱스 :', id_tokens)
# [결과]
# 전체를 본 후 공백을 포함한다음 공백에 대한 정보들이 토큰으로 분리된 것을 확인할 수 있다.
# 공백에 대한 정보는 유니코드 형식으로 ▁(언더바)로 표시된다.

# 디코딩 (원문으로 복원)
print("".join(tokens).replace("▁", " ").strip())  # 토큰을 다시 원문으로 복원
print(sp.decode_pieces(tokens))  # 토큰을 다시 원문으로 복원
print(sp.decode_ids(id_tokens))  # 토큰 인덱스를 다시 원문으로 복원

원문 : 걸작은 몇안되고 졸작들만 넘쳐난다.
토큰화 : ['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
인덱스 : [1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.


### BertWordPieceTokenizer
- 

In [24]:
!pip install tokenizers

In [ ]:
from tokenizers import BertWordPieceTokenizer

# 학습할 파일이 한국어로 되어 있어서 기본 설정을 변경
tokenizer = BertWordPieceTokenizer(
    lowercase=False,  # 소문자 변환 여부
    strip_accents=False # 악센트 제거 여부, 발음 강세문자 기호 제거 여부 (예: café -> cafe)
    )  
vocab_size = 10000

tokenizer.train(
    files=["naver_review.txt"],  # 학습할 파일 경로
    # => (전처리된 파일을 넘겨주면 비지도 학습이지만 내부적으로 자기 지도 학습을함.)
    vocab_size=vocab_size,  # 단어 집합의 크기
    min_frequency=5,  # 최소 단어 빈도
    show_progress=True,  # 학습 진행 상황 표시 
    )

In [26]:
# 학습된 토크나이저 저장
# tokenizer.save_model() : 토크나이저 모델을 지정한 경로에 저장
tokenizer.save_model('./','bert_word_piece_from_naver_review') 

# [결과] : bert_word_piece_from_naver_review-vocab.txt 파일이 생성됨

['./bert_word_piece_from_naver_review-vocab.txt']

In [31]:
text = ratings_test_path_df['document'][100]  # 테스트 데이터의 100번째 문장
encoded = tokenizer.encode(text)  # 문장 인코딩
print(text) # 원문 출력
print(encoded) # Encoding 객체가 출력됨
print('토큰화 :', encoded.tokens)  # 토큰화 결과 출력
print('인덱스 :', encoded.ids)  # 토큰 인덱스 결과 출력

# [결과]
# 앞에 뭐가 붙는지 확인하는 의미로 (##) 표시가 붙음
# 공백에 대한 정보는 토큰으로 분리되지 않음

print('디코딩 :', tokenizer.decode(encoded.ids))  # 인덱스를 다시 원문으로 디코딩


걸작은 몇안되고 졸작들만 넘쳐난다.
Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
토큰화 : ['걸작', '##은', '몇', '##안되고', '졸작', '##들만', '넘쳐', '##난다', '.']
인덱스 : [2759, 1123, 445, 9507, 2589, 3799, 8334, 2430, 16]
디코딩 : 걸작은 몇안되고 졸작들만 넘쳐난다.


---
### 전처리 연습

1. 적절한 데이터셋을 찾거나 생성한다.
2. 적절한 전처리를 진행한다.
3. TfidfVectorizer를 이용하여 벡터화 한다. (중심 벡트를 잡는다.)
4. Cosine Similarity를 계산하여 입력된 문자열의 긍/부정을 판단한다.

In [39]:
# 1. 적절한 데이터셋 생성.
texts = 'The Matrix is everywhere its all around us, here even in this room. \
You can see it out your window or on your television. \
You feel it when you go to work, or go to church or pay your taxes.' 

In [40]:
# 2. 적절한 전처리 진행.
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))

# 데이터 전처리 함수 임포트
from text_preprocessing import (
    preprocess_text_for_encoding, 
)
vocab, preprocessed_sentences = preprocess_text_for_encoding(texts)

print(preprocessed_sentences)  # 전처리된 문장들 출력

[['matrix', 'everywhere', 'around', 'even', 'room'], ['see', 'window', 'television'], ['feel', 'work', 'church', 'pay', 'taxes']]
